In [1]:
import geopandas as gpd
import pandas as pd

glow_width = pd.read_csv('/home/xj/device/data/glow/GLOW_Width/GLOW_width_region_4.csv')
print(glow_width.columns)
polygon_match = gpd.read_file('/home/xj/device/data/glow/GLOW_Cross_section/GLOW_crosssection_region_4.shp')
print(polygon_match.columns)

Index(['ID', 'date', 'width'], dtype='object')
Index(['ID', 'lat', 'lon', 'geometry'], dtype='object')


In [2]:
glow_points = polygon_match 
matched_polygons = gpd.read_file('../1-buffer-match/600mRiverReach-straightBuffer-corrected-20station.shp')
matched_polygons = matched_polygons.to_crs(glow_points.crs)

# 进行空间连接，保留在 polygon 内的点
points_in_polygons = gpd.sjoin(glow_points, matched_polygons, how="inner", predicate="within")

points_in_polygons.rename(columns={
    'lat_left': 'glow_lat',
    'lon_left': 'glow_lon',
    'lat_right':'gage_lat',
    'lon_right':'gage_lon',
    # 'distance':'GRGEpoints-distance-COMID',
    'width':'COMID-meanwidth',
}, inplace=True)

# 保存结果到新的 Shapefile
points_in_polygons.to_file("1.north_GLOW_buffer_matched_points.shp")
print(points_in_polygons.columns)

Index(['ID', 'glow_lat', 'glow_lon', 'geometry', 'index_right', 'COMID', 'id',
       'Width', 'gage_lat', 'gage_lon', 'STCD', 'stnmpy', 'Width_deg'],
      dtype='object')


/tmp/ipykernel_77907/1268166301.py:18: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  points_in_polygons.to_file("1.north_GLOW_buffer_matched_points.shp")


In [3]:
import geopandas as gpd
import pandas as pd

glow_width = pd.read_csv('/home/xj/device/data/glow/GLOW_Width/GLOW_width_region_4.csv')
points_in_polygons = gpd.read_file("1.north_GLOW_buffer_matched_points.shp")
points_in_polygons = points_in_polygons.rename(columns={'stnmpy':'stationid'})
att_csv = points_in_polygons.drop(columns=["geometry"])
width_timeseries = att_csv[['ID','stationid']].merge(glow_width, on = 'ID', how = 'right')
width_timeseries.to_csv("1.north_glow_width_timeseries.csv")
print(width_timeseries.columns)

average_width_df = width_timeseries.groupby(['stationid','date'])['width'].mean().reset_index()
average_width_df.to_csv("1.north_glow_datemean_width_timeseries.csv", index=False)

Index(['ID', 'stationid', 'date', 'width'], dtype='object')
